In [1]:
# ライブラリのインストール
# ちなみにCPUでもfaiss-gpuは動く
!pip install -Uqq sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

In [33]:
def search_nearest_sentence(sentence_refs, sentence_queries, model_name="intfloat/multilingual-e5-base", path_out_faiss_full_index_path=None):
    # モデルの選択と読み込み
    model = SentenceTransformer(model_name)

    # 英語ならば他にもよいモデルがある
    # https://huggingface.co/spaces/mteb/leaderboard など参照
    # model_name = "thenlper/gte-base"
    # model = SentenceTransformer(model_name)

    # データのエンコード
    embeddings_refrence = model.encode(
        sentence_refs,
        normalize_embeddings=True,
        show_progress_bar=True,
        convert_to_tensor=True
    )

    embeddings_query = model.encode(
        sentence_queries,
        normalize_embeddings=True,
        show_progress_bar=True,
        convert_to_tensor=True
    )

    # Faiss インデックスの構築と保存
    # faiss.IndexFlatL2の初期設定として、次元数を設定
    faiss_index = faiss.IndexFlatL2(len(embeddings_refrence[0]))
    faiss_index.add(embeddings_refrence.detach().cpu().numpy())

    if path_out_faiss_full_index_path is not None:
        faiss.write_index(faiss_index, path_out_faiss_full_index_path)

    # 類似度検索と結果の保存
    # 今回は1個だけ検索という前提のため1
    search_score, idx_list = faiss_index.search(embeddings_query.detach().cpu().numpy().astype(np.float32), 1)

    df_out = pd.DataFrame([search_score.flatten(), idx_list.flatten()]).T
    df_out.columns = ["Score", "ID"]
    df_out["ID"] = df_out["ID"].astype(int)
    df_out["Sentence"] = [sentence_refs[idx] for idx in df_out["ID"]]

    return df_out


In [34]:
# 0個目は似た意味の文書を入れた
# その他はzennの記事からlistとして
article_titles = [
    "Vector Searchによる検索",
    "Firebase Authのリダイレクトログインを使っている人は今年の6月までに対応しないと大変ですよという注意喚起",
    "データ分析基盤まとめ(随時更新)",
    "Claude3にプロジェクト全体をぶち込むためのプロジェクトの構造とファイル内容を自動でまとめるPythonスクリプト",
    "モダンなタスク管理を可能にするSnowflake Python API",
    "Vercelで配信していた静的ページをVite + Cloudflare Pagesに置き換えた",
    "新しいMacBookのセットアップ",
    "cloudflare-workers で動く claude3 の discord-bot を作ってみた",
    "terraform state 理解してないのにterraform apply している人、危機感持った方が良いって",
    "Webエンジニアになって1年経ったので、これまでやってきたことをまとめる。",
    "PyAirbyteで始める簡単Data Injest Pipeline",
    "Terraform v1.8.0でリリースされそうなプロバイダ独自関数を試す",
    "Claude 3 opusの画像認識性能をお試し",
    "【Flutter】ClipPath と CustomClip を使って身分証撮影ガイドを作成する",
    "初心者がまとめた初心者のためのコマンドライン操作",
    "Reactで使えそうなグラフ描画ツールってなぁになぁに？",
    "NestJSでCloud Loggingのための構造化ロギング",
    "Astro DBをAstro以外で使う",
    "【Vue.js】イメージとアナロジーとヒストリーで理解する Component Emits",
    "[App Store Connect] デジタルサービス法のコンプライアンスの警告対応と解釈について",
    "Notionで全ての仕事を管理する方法を徹底解説",
    "Aptosを使ってたまごっちライクなブロックチェーンゲームを作ってみよう！！ 〜Part 2〜",
    "Flutter StateError (Bad state: Type mismatch between hooks:",
    "【Rails】Brakemanでセキュリティチェックをする",
    "【Typescript5.4】NoInferを活用して型安全なUIコンポーネントを設計する",
    "VirtualBoxでOracle Database 19c RACを構築（前編）",
    "timm.create_modelの実装の解説",
    "log4jの0-day exploitを動かして理解する",
    "[GAS]職員割当表を作成する",
    "Next.js + MUI で簡単な管理画面テンプレートを作ってみる(その１)",
    "拡散モデルと表データ生成②：【論文】CoDi",
    "OAuth2.0はなぜ複雑に見えるのか",
    "Device Preview for Flutter、githubaction,firebasehostringを使って爆速レビューを目指す",
    "Railsのin_batchesを読み解く",
    "Neovimがtreesitter parser errorを出すようになってしまったのでworkaround",
    "Chakra UI で Menu や Popover で表示される謎のスペースをなくす方法",
    "マルチプロセスなBun.serveの並列リクエスト処理性能を調べる",
    "【Python】実践データ分析100本ノック　第5章",
    "Raspberry PiでSlashGPTを動かす",
    "【Microsoft Fabric】 データフロー Gen2 について",
    "世界初？Claude3を使った動画解析 - claude3-video-analyzer",
    "FlutterパッケージのPrivacy Manifests対応バージョン",
    "Atcoder - ABC345 A - D まで復習解説",
    "Terraformでマルチクラウド環境の認証情報をひとつにまとめる",
    "goに入門したのでgcのDOCを読んだメモ",
    "Developer登録から３カ月で個人開発者がビジネス提携を持ちかけられた話",
    "永久機関が完成しちまったな～！",
    "ASUS「Zenfone 11 Ultra」のプロセッサが全然フツーじゃなかった件",
    "Raspberry Pi 4とBuildrootでRAUCを使ってアップデート環境を作ってみる",
]

In [35]:
search_nearest_sentence(article_titles, ["似た文書をベクトル検索で探し出したい ~SentenceTransformersとFaissで効率的にベクトル検索~"])

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,Score,ID,Sentence
0,0.278252,0,Vector Searchによる検索
